In [1]:
%load_ext autotime

import numpy as np
import pandas as pd

import scrapy
from scrapy.crawler import CrawlerRunner

from crochet import setup, wait_for

import requests

import json
import re
setup()

time: 1.32 s (started: 2021-09-28 12:55:40 +07:00)


In [5]:
class ScheduleSpider(scrapy.Spider):
    name = 'schedule'

    def start_requests(self):
        username = '518H0003'
        passw = ''

        session = requests.Session()
        response = session.post("https://stdportal.tdtu.edu.vn/Login/SignIn?ReturnURL=https://stdportal.tdtu.edu.vn/",
                                data={'user': username, 'pass': passw}, cookies=session.cookies.get_dict())

        ssoUrl=json.loads(response.text)['url']
        token = re.findall('(?<=Token=)([^&]*)(?=&)?',ssoUrl)

        urls = [
            'http://sso.tdt.edu.vn/Authenticate.aspx?ReturnUrl=https://lichhoc-lichthi.tdtu.edu.vn/tkb2.aspx&Token='+token[0]
        ]

        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        #get headrow
        cols = len(response.xpath('//tr[contains(@class,\'Headerrow\')]/td'))
        rows = len(response.xpath('//tr[contains(@class,\'rowContent\')]'))

        arrHeads = []
        for head in response.xpath('//tr[contains(@class,\'Headerrow\')]/td'):
            h = head.xpath('text()').extract()
            tmp = re.findall(r'\((.*)\)',h[1])
            if len(tmp) > 0:
                arrHeads.append(tmp[0])

        arrHeads = np.array(arrHeads).ravel()

        #crawl item
        arrItems = np.zeros((rows, cols - 1), dtype='U256')

        for r in range(1,rows):
            for c in range(1,cols):
                ptr = '//tr[contains(@class,\'rowContent\')]['+str(r)+']/td['+str(c)+']/table'
                if(len(response.xpath(ptr))>0):
                    crawl = response.xpath(ptr+'/tr/td/b/text()').extract()
                    arrItems[r - 1][c - 2] = crawl[0]

        dfSchedule = pd.DataFrame(arrItems, columns=arrHeads)

        with open('Output/ScrapySchedule.txt', 'w+') as f:
            for i in dfSchedule.columns:
                f.write(f'Ngay thang: {i} \n')
                if (dfSchedule[i].sum() == ''):
                    f.write(f'Lich cong tac: Khong co \n')
                else:
                    for j in dfSchedule[i]:
                        if (j != ''):
                            f.write(f'Lich cong tac: {j} \n')
                f.write(f'\n')

time: 4.08 ms (started: 2021-09-28 12:56:20 +07:00)


In [3]:
@wait_for(30)
def run_spider():
    crawler = CrawlerRunner({
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
    })
    d = crawler.crawl(ScheduleSpider)
    return d

time: 1.04 ms (started: 2021-09-28 12:55:44 +07:00)


In [4]:
run_spider()



time: 2.53 s (started: 2021-09-28 12:55:46 +07:00)
